In [ ]:
import numpy as np
import math

class hyperband:

    def __init__(self,space):
        """ The search space components, initialised by the user """

        self.search_space = space



    def __sample(self,n_samples):

        """ It draws a sample 'n_samples' times from the search space uniformly

            Arguments 
              n_samples : no. of points to sample from the search space
                        dtype - int

            Returns
              config : sampled hyperparameter configurations
                        dtype - numpyarray        
        """

        config = np.array([np.random.choice(self.search_space[val],n_samples,replace=True) for val in self.search_space.keys()])
        return np.transpose(config)


    def search(self,model_definition,max_iter=81,eta=3,skip_last=1,using_loss=False):
      
        """ Assigns the number of unique brackets, number of resources and configs, implements the successive halving
        
            Arguments
              model_definition : a user defined function which contains the definition of the model
                                
              max_iter : maximum no. of iterations allowed for one hyper-parameter configuration,    dtype - int    default - 81

              eta : reduction rate of configuration in each successive halving,   dtype - int   default - 3

              skip_last : if 1 skips the last bracket and vice-versa,   dtype - int  default - 1

              using_loss : if true uses loss function of the model to evaluate in sucessive halving and if not uses val-metric of the model,   dtype - bool   default - False


            Returns
              best : the best hyper-parameter configuration 
        """

        logeta = lambda x: math.log(x) / math.log(eta)
        s_max = int(logeta(max_iter))
        B = (s_max + 1) * max_iter

        result = np.array([])
        best_config = np.array([])

        ## this loop denotes the no. of unique run of successive halving
        for s in reversed(range(s_max + 1)):

            print('\n  Current bracket number - ', s)

            if skip_last:
                if s == 0: break

            n = int(math.ceil(int(B / max_iter / (s + 1)) * eta ** s))  # number of configurations to sample for at starting of given bracket
            r = max_iter * eta ** (-s)  # number of resources at starting for given bracket

            T = self.__sample(n)  # sampling from the search space
            metric = np.array([])

            ## this loop runs the successive halving for a given bracket
            for i in range(s + 1):

                n_i = n * eta ** (-i)  # no. of configs for given successive halving
                r_i = r * eta ** (i)  # no. of resources for given successive halving
                val_metric = np.array([model_definition(r_i,t) for t in T])  # getting the val_metric for each config
                if using_loss: T = np.array([T[i] for i in reversed(np.argsort(val_metric)[int(n / eta):])])  # implementing the successive halving
                else: T = np.array([T[i] for i in np.argsort(val_metric)[:int(n / eta)]])
                metric = np.append(metric,np.max(val_metric))

                print('\n\n \t number of reduction/successive halving done - ', i)

            best_config = np.append(best_config, T[0])  # keeping track of the best config from each bracket
            result = np.append(result,metric[-1])

        best = best_config[np.argmax(result)]
        print('\n\n the best configuration - ', best)

        return best  # return the best config from all the brackets by max val_metric



In [ ]:
## defining the search space

space = {'lr': np.array([0.00001,0.00005,0.00008,0.0001,0.0005,0.0008,0.001,0.005,0.008,0.01]),
                  'beta1': np.arange(0.86,0.94,0.02),
                  'beta2': np.arange(0.997,0.999,0.001),
                  'loss': np.array(['focal_loss','cross_entropy'])}
hyp = hyperband(space)

In [ ]:
## since I'm using it for segmentation, I've defined my own models,loss functions and metrics

from keras.models import Model
from keras.layers import Add,Input,MaxPooling2D,concatenate,BatchNormalization
from keras.regularizers import l2
from keras.applications import *
from keras.layers.convolutional import Conv2D,Conv2DTranspose
from keras.layers.core import Activation


class UNET:

    class unet:
        """ Unet architecture

               Usage :
                   unet = Semantic_Segmentation.models.UNET.unet(input_shape,no_classes,regularizer,summary)
                   model = unet.build()

               # Arguments
                   input_shape : size of the input image ,in tuple.
                   n_classes : the number of target class.
                               dtype --> int
                   regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                               dtype --> float   default -->None
                   summary : If True prints the model summary
                                default --> True
           """
        build = lambda self: self.__architecture()
        weight_decay = lambda self, x: None if x == None else l2(x)
        batchnorm = lambda self, x: BatchNormalization(beta_regularizer=l2(0.001),
                                                       gamma_regularizer=l2(0.001)) if x else Activation('linear')

        def __init__(self, input_shape, n_classes, regularizer=None, BatchNorm=True, summary=True):
            self.input_shape = input_shape
            self.n_classes = n_classes
            self.regularizer = regularizer
            self.BatchNorm = BatchNorm
            self.summary = summary

        def __architecture(self):
            input = Input(self.input_shape)

            conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(input)
            conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv1)
            batchnorm1 = self.batchnorm(self.BatchNorm)(conv1)
            pool1 = MaxPooling2D(pool_size=(2, 2))(batchnorm1)

            conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(pool1)
            conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv2)
            batchnorm2 = self.batchnorm(self.BatchNorm)(conv2)
            pool2 = MaxPooling2D(pool_size=(2, 2))(batchnorm2)

            conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(pool2)
            conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv3)
            batchnorm3 = self.batchnorm(self.BatchNorm)(conv3)
            pool3 = MaxPooling2D(pool_size=(2, 2))(batchnorm3)

            conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(pool3)
            conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv4)
            batchnorm4 = self.batchnorm(self.BatchNorm)(conv4)
            pool4 = MaxPooling2D(pool_size=(2, 2))(batchnorm4)

            conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(pool4)
            conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv5)
            batchnorm5 = self.batchnorm(self.BatchNorm)(conv5)
            up5 = Conv2DTranspose(512, 4, strides=(2, 2), padding='same',
                                  kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_5')(batchnorm5)

            merge6 = concatenate([batchnorm4, up5], axis=3)
            conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(merge6)
            conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv6)
            batchnorm6 = self.batchnorm(self.BatchNorm)(conv6)
            up6 = Conv2DTranspose(256, 4, strides=(2, 2), padding='same',
                                  kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_6')(batchnorm6)

            merge7 = concatenate([batchnorm3, up6], axis=3)
            conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(merge7)
            conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv7)
            batchnorm7 = self.batchnorm(self.BatchNorm)(conv7)
            up7 = Conv2DTranspose(128, 4, strides=(2, 2), padding='same',
                                  kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_7')(batchnorm7)

            merge8 = concatenate([batchnorm2, up7], axis=3)
            conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
            conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
            batchnorm8 = self.batchnorm(self.BatchNorm)(conv8)
            up8 = Conv2DTranspose(64, 4, strides=(2, 2), padding='same',
                                  kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_8')(batchnorm8)

            merge9 = concatenate([batchnorm1, up8], axis=3)
            conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(merge9)
            conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv9)
            conv9 = Conv2D(self.n_classes, 1, activation='softmax', padding='same', kernel_initializer='he_normal',
                           kernel_regularizer=self.weight_decay(self.regularizer))(conv9)
            #op = Activation('softmax', name='softmax')(conv9)

            model = Model(input=input, output=conv9, name='Unet')
            if self.summary: print(model.summary())
            return model


class FCN:
    """ Available models --> fcn32, fcn16, fcn8

        All the three avilable with vgg16,vgg19,resnet50
    """

    class fcn32:

        """ FCN32 architecture, it doesn't use any skip connections and upsamples the image by the scale of 32.

            Usage :
                FCN32 = Semantic_Segmentation.models.FCN.fcn32(input_shape,no_classes,base_model_name,pretrained_weights,regularizer,summary)
                model = FCN32.build()

            # Arguments
                input_shape : size of the input image ,in tuple.
                n_classes : the number of target class.
                            dtype --> int
                base_model : name of the pre-trained cnn model on top of which FCN is built.
                            dtype --> string  default --> 'vgg16'
                weight_path : path to the pre_trained weight files.
                            dtype --> string  default --> 'imagenet'
                regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                            dtype --> float   default -->None
                summary : If True prints the model summary
                                default --> True
        """
        weight_decay = lambda self,x: l2(x) if type(x)==int else None
        build = lambda self: self.__decoder(self.encoder())

        def __init__(self,input_shape,n_classes,base_model='vgg16',weight_path='imagenet',regularizer = None,summary=True):
            self.base_model = base_model
            self.input_shape = input_shape
            self.n_classes = n_classes
            self.weightpath = weight_path
            self.regularizer = regularizer
            self.summary = summary


        def encoder(self):

            """ Builds the encoder layer on top of the given pre-trained CNN model

                # Returns
                    model_en : the architecture of the encoder part
            """
            if self.base_model == 'vgg16':
                base = vgg16.VGG16(include_top=False, weights=self.weightpath, pooling=None, input_shape=self.input_shape)
            elif self.base_model == 'vgg19':
                base = vgg19.VGG19(include_top=False, weights=self.weightpath, pooling=None, input_shape=self.input_shape)
            else: base = resnet50.ResNet50(include_top=False, weights=self.weightpath, pooling=None, input_shape=self.input_shape)

            layer_encoder = Conv2D(4096, (7, 7), padding='same', activation='relu', kernel_regularizer=self.weight_decay(self.regularizer),
                                   name='conv_en1')(base.output)
            layer_encoder = Conv2D(4096, (1, 1), padding='same', activation='relu', kernel_regularizer=self.weight_decay(self.regularizer),
                                   name='conv_en2')(layer_encoder)

            encoder_model = Model(base.input,layer_encoder)
            return encoder_model


        def __decoder(self,model_en):

            """ Builds the decoder layer on top of the encoder part.

                # Arguments
                    model_en : the model architecture of the encoder part

                # Returns
                    model : the full model architecture, both encoder and decoder combined
            """
            layer_decoder = Conv2D(self.n_classes, (1,1), padding='same', activation='relu', kernel_regularizer=self.weight_decay(self.regularizer),
                                   name = 'conv_dec')(model_en.output)
            layer_decoder = Conv2DTranspose(self.n_classes, (64,64), strides=(32,32), padding = 'same', kernel_regularizer = self.weight_decay(self.regularizer),
                                            name = 'deconv')(layer_decoder)
            output = Activation('softmax',name='softmax')(layer_decoder)

            model = Model(model_en.input,output,name='FCN32-'+self.base_model)
            if self.summary: print(model.summary())
            return model


        def skip_connections(self,model_en):

            """ Builds the skip connections based on the pre-trained CNN models for FCN16 and FCN8.
                NOTE- FCN32 doesnt use this function, its a base for other class/FCN16,FCN8

                # Arguments:
                    model_en : the model architecture of the encoder part
            """
            if self.base_model == 'resnet50': skip_1,skip_2 = model_en.get_layer(model_en.layers[112].name).output, model_en.get_layer(model_en.layers[50].name).output
            else: skip_1,skip_2 = model_en.get_layer('block4_pool').output, model_en.get_layer('block3_pool').output
            return skip_1,skip_2



 


    class fcn16(fcn32):

        """ FCN16 architecture, it uses one skip connection and upsamples the image by the scale of 16.

            Usage :
                FCN16 = Semantic_Segmentation.models.FCN.fcn16(input_shape,no_classes,base_model_name,pretrained_weights,regularizer,summary)
                model = FCN16.build()

            # Arguments
                input_shape : size of the input image ,in tuple.
                n_classes : the number of target class.
                            dtype --> int
                base_model : name of the pre-trained cnn model on top of which FCN is built.
                            dtype --> string  default --> 'vgg16'
                weight_path : path to the pre_trained weight files.
                            dtype --> string  default --> 'imagenet'
                regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                            dtype --> float   default -->None
                summary : If True prints the model summary
                                default --> True
        """

        ## NOTE : It wraps the properties of FCN32, same arguments and encoder part but it uses diff decoder function
        ##since the decoder includes skip connection.
        build = lambda self: self.__decoder(super().encoder())

        def __decoder(self,model_en):

            skip,_ = super().skip_connections(model_en)

            layer_decoder = Conv2DTranspose(skip.get_shape().as_list()[-1], (4, 4), strides=(2, 2), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv1')(model_en.output)
            layer_decoder = Add(name='skip1')([layer_decoder, skip])
            layer_decoder = Conv2DTranspose(self.n_classes, (32,32), strides=(16, 16), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv3')(layer_decoder)
            output = Activation('softmax',name='softmax')(layer_decoder)

            model = Model(model_en.input, output,name='FCN16-'+self.base_model)
            if self.summary: print(model.summary())
            return model




    class fcn8(fcn32):

        """ FCN8 architecture, it uses two skip connections and upsamples the image by the scale of 8, preserves most of the information.

            Usage :
                FCN8 = Semantic_Segmentation.models.FCN.fcn8(input_shape,no_classes,base_model_name,pretrained_weights,regularizer,summary)
                model = FCN8.build()

            # Arguments
                input_shape : size of the input image ,in tuple.
                n_classes : the number of target class.
                            dtype --> int
                base_model : name of the pre-trained cnn model on top of which FCN is built.
                            dtype --> string  default --> 'vgg16'
                weight_path : path to the pre_trained weight files.
                            dtype --> string  default --> 'imagenet'
                regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                            dtype --> float   default -->None
                summary : If True prints the model summary
                                default --> True
        """

        ## NOTE : It wraps the properties of FCN32, same arguments and encoder part but it uses diff decoder function
        ##since the decoder includes 2 skip connections.

        build = lambda self: self.__decoder(super().encoder())

        def __decoder(self,model_en):

            skip_1,skip_2 = super().skip_connections(model_en)

            layer_decoder = Conv2DTranspose(skip_1.get_shape().as_list()[-1], (4, 4), strides=(2, 2), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv1')(model_en.output)
            layer_decoder = Add(name='skip1')([layer_decoder, skip_1])
            layer_decoder = Conv2DTranspose(skip_2.get_shape().as_list()[-1], (4, 4), strides=(2, 2), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv2')(layer_decoder)
            layer_decoder = Add(name='skip2')([layer_decoder, skip_2])
            layer_decoder = Conv2DTranspose(self.n_classes, (16, 16), strides=(8, 8), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv3')(layer_decoder)
            output = Activation('softmax',name='softmax')(layer_decoder)

            model = Model(model_en.input,output,name='FCN8-'+self.base_model)
            if self.summary: print(model.summary())
            return model




class focal_loss:

    """ A loss function similar to cross_entropy

        # Usage
            model.compile('sgd',loss=focal_loss.loss,.......)

        # Arguments
            class_weights : weights for each class to solve the class imbalance problem.
                            dtype --> array   default --> None
            pixel_weights : weights for each pixels in order to segment certain part of the image clearly.
                            dtype --> array   default --> None
    """
    def c_weights(self,x):
      try:
        if list(x)!=None: return x
      except TypeError: return int('1')

    def p_weights(self,x):
      try:
        if list(x)!=None: return x
      except TypeError: return int('1')  



    clipping = lambda self,x: K.clip(x, K.epsilon(), 1.-K.epsilon())


    def __init__(self,class_weights=None, pixel_weights=None, gamma=2):
        self.class_weights = class_weights
        self.gamma = gamma
        self.pixel_weights = pixel_weights

    def loss(self,y_true,y_pred):

        """ executes the focal loss

            # Arguments
                y_true : true class values
                y_pred : predicted class values from the model
            # Returns
                fl : mean focal loss for the given batch
         """
        y_pred = self.clipping(y_pred)
        fl = -(K.sum((self.c_weights(self.class_weights) * K.pow(1.-y_pred,self.gamma) * (y_true * K.log(y_pred))),axis=-1))
        fl = K.sum((self.p_weights(self.pixel_weights) * fl),axis=(1,2))
        fl = K.mean(fl, axis=0)
        return fl/100



class cross_entropy(focal_loss):

    """ Categorical cross_entropy
        NOTE : for binary classification it uses softmax instead sigmoid

        # Usage
            model.compile('sgd',loss=cross_entropy.loss,.......)

        # Arguments
            class_weights : weights for each class to solve the class imbalance problem.
                            dtype --> array   default --> None
            pixel_weights : weights for each pixels in order to segment certain part of the image clearly.
                            dtype --> array   default --> None
    """

    ## NOTE - this class inherits the properties of focal_loss class
    def loss(self,y_true,y_pred):

        """ executes the categorical cross-entropy

            # Arguments
                y_true : true class values
                y_pred : predicted class values from the model
            # Returns
                ce : mean cross-entropy for the given batch
        """
        y_pred = super().clipping(y_pred)
        ce = -(K.sum((super().c_weights(self.class_weights) * (y_true * K.log(y_pred))),axis=-1))
        ce = K.sum((super().p_weights(self.pixel_weights) * ce),axis=(1,2))
        ce = K.mean(ce,axis=0)
        return ce/100



class dice_loss(focal_loss):

    """ Its similar to IoU, dice_coeff = (2*A^B)/A U B  dice_loss= 1- dice_coeff
        # Usage
            model.compile('sgd',loss=dice_loss.loss,.......)

        # Arguments
            class_weights : weights for each class to solve the class imbalance problem.
                            dtype --> array   default --> None
            pixel_weights : weights for each pixels in order to segment certain part of the image clearly.
                            dtype --> array   default --> None
    """

    ## NOTE - this class inherits the properties of focal_loss class
    def loss(self,y_true,y_pred):

        """ executes the dice loss

            # Arguments
                y_true : true class values
                y_pred : predicted class values from the model
            # Returns
                dl : dice loss for the given batch
        """
        y_pred = super().clipping(y_pred)
        intersection = K.sum((super().p_weights(self.pixel_weights) * y_true * y_pred),axis=(1,2))
        intersection = K.sum((super().c_weights(self.class_weights) * intersection),axis=-1)
        union = K.sum( (super().p_weights(self.pixel_weights)*(y_true + y_pred) ),axis=(1,2))
        union = K.sum((super().c_weights(self.class_weights) * union),axis=-1)
        dl = (1. - ((2*intersection)/union))
        return K.mean(dl)


class Metrics:

    __weighted_method = lambda self,x,y,string,w: (K.sum(x)/K.sum(y)) if string=='inter' else (K.sum(w*x)/K.sum(w*y))
    __avg_method = lambda self,x,y,string,w: K.mean(x/y) if string=='intra' else self.__weighted_method(x,y,string,w)


    def __metrics_base(self,y_true,y_pred):

        """ Base for all the metrics defined below """
        y_true,y_pred = K.flatten(tf.math.argmax(y_true,axis=-1)), K.flatten(tf.math.argmax(y_pred,axis=-1))
        con_mat = K.cast(tf.math.confusion_matrix(y_true,y_pred), K.floatx())
        correct = tf.linalg.diag_part(con_mat)
        total = K.sum(con_mat, axis=-1)
        return correct,total,con_mat


    def accuracy(self,y_true,y_pred):

        """ computes the accuracy

            # Arguments
                y_true : target value
                y_pred : predicted class value
            # Returns
                acc : overall accuracy
        """
        correct,total,_ = self.__metrics_base(y_true,y_pred)
        return ( K.sum(correct) / K.sum(total) )


    def IoU(self, y_true, y_pred,average='inter',weights=None):

        """ Intersection over Union , IoU = A^B/(A U B - A^B)
           Computes the percentage overlap with the target image.

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the IoU score overall  'intra' --> computes the score for each calss and computes the average
                        'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                IoU score
        """
        _, _, con_mat = self.__metrics_base(y_true, y_pred)
        intersection = tf.linalg.diag_part(con_mat)
        ground_truth_set = K.sum(con_mat, axis=1)
        predicted_set = K.sum(con_mat, axis=0)
        union = ground_truth_set + predicted_set - intersection
        return self.__avg_method(intersection,union,average,weights)


    def recall(self,y_true,y_pred,average='inter',weights=None):

        """ Computes the recall score over each given class and gives the overall score.  recall = TP/TP+FN

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the recall score overall  'intra' --> computes the score for each calss and computes the average
                        'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                recall score
        """
        correct,total,_ = self.__metrics_base(y_true,y_pred)
        return self.__avg_method(correct,total,average,weights)


    def precision(self,y_true,y_pred,average='inter',weights=None):

        """ Computes the precision over each given class and returns the overall score.  precision = TP/TP+FP

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the precision score overall  'intra' --> computes the score for each calss and computes the average
                        'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                precision score
        """
        correct,_,con_mat = self.__metrics_base(y_true,y_pred)
        total = K.sum(con_mat,axis=0)
        return self.__avg_method(correct,total,average,weights)


    def f1score(self,y_true,y_pred,average='inter',weights=None):

        """ Computes the f1 score over each given class and returns the overall score.  f1 = (2*precision*recall)/(precision+recall)

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the f1 score overall  'intra' --> computes the score for each calss and computes the average
                            'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                 f1 score
        """
        precision = self.precision(y_true,y_pred,average,weights)
        recall = self.recall(y_true,y_pred,average,weights)
        return ((2*precision*recall)/(precision+recall))



    def dice_coeffiecient(self,y_true,y_pred,average='inter',weights=None):
        """ Computes the dice score over each given class and returns the overall score.

                # Arguments
                    y_true : target value
                    y_pred : predicted class value
                    average : 'inter' --> computes the dice score overall  'intra' --> computes the score for each calss and computes the average
                                    'weighted' --> computes the weighted average , useful for imabalanced class.
                    weights :  only if average is specified 'weighted', weights for the respective classes.
                # Returns
                    dice score
                """

        y_pred = focal_loss.clipping(y_pred)
        intersection = 2 * K.sum((y_true * y_pred),axis=(0,1,2))
        union = K.sum( (y_true*y_true) + (y_pred*y_pred),axis=(0,1,2))
        return self.__avg_method(intersection,union,average,weights)

Using TensorFlow backend.


In [ ]:
## No need to look, its my own function to convert into onehots

def data_convert(data1,data2,data3,data4):

        ## the function to get one-hot encoded values
        #print('\n converting the images.... \n')
        for i in range(12): data4[(np.where((np.sum(abs(data1 - data2[i]), axis=-1)) == 0))] = data3[i]
        return data4

In [ ]:
import tensorflow as tf
import cv2
import glob
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
## no need to look, its for getting the data from drive

path_y = "/content/drive/My Drive/dataset/filtered data/seg_f/"
path_x = "/content/drive/My Drive/dataset/filtered data/rgb_f/"

n_classes = 12


## color map instruction
label_value = np.array([[0,0,0],[70,70,70],[153,153,190],[160, 170, 250],[153, 153, 153],[50, 234, 157],[128, 64, 128],[232, 35, 244],[35, 142, 107],[142, 0, 0],[156, 102, 102],[0, 220, 220]])
label_class = np.array(['Unlabeled','Building','Fence','Other','Pole','Road line','Road','Sidewalk','Vegetation','Car','Wall','Traffic sign'])


## reading the data from image
data_y = np.array([cv2.imread(filename,cv2.IMREAD_COLOR) for filename in glob.glob(path_y+'*.png')])
data_x = np.array([cv2.imread(filename,cv2.IMREAD_COLOR) for filename in glob.glob(path_x+'*.png')])
 
print(data_x.shape)
print(data_y.shape)

(471, 320, 320, 3)
(471, 320, 320, 3)


In [ ]:
train_x,train_y = data_x[0:20],data_y[0:20]
val_x,val_y = data_x[21:25],data_y[21:25]


data_y_1hot = np.zeros([train_y.shape[0], train_y.shape[1], train_y.shape[2], 12])
data1hot = np.zeros([val_y.shape[0], val_y.shape[1], val_y.shape[2], 12])
label_1hot = np.identity(12)
train_y_1hot = data_convert(train_y, label_value, label_1hot, data_y_1hot)
val_y_1hot = data_convert(val_y, label_value, label_1hot, data1hot)


 

In [ ]:
## this is important, the model_definition function which is used in the hyperband
## I've defined my model here and used the configurations given from hyperband for each iteration 

def define(res,params):
        '''
          Arguments
            params : one hyper-parameter configuration,  dtype - numpyarray

            res : number of epochs to run the model,   dtype - int
        
          Returns
            metric : the validation metric for the corresponding configuration
        '''

        model = UNET.unet((320,320,3),n_classes,BatchNorm=False,summary=False)
        model = model.build()

        print('\n \t running the config ',params,' for - ',int(res),' epochs')
        
        l = lambda x:focal_loss().loss if x == 'focal_loss' else cross_entropy().loss    ## since loss function is HP, im using lambda function to choos b/w diff loss functions
        
        ## compiling the model as per the given hyperparameter config
        optimizer = Adam(learning_rate=float(params[0]),beta_1=float(params[1]),beta_2=float(params[2]))
        model.compile(optimizer,loss = l(params[3])  ,metrics=[Metrics().IoU])

        ## train the model
        train = model.fit(train_x,train_y_1hot,epochs = int(res),batch_size = 8,validation_data=(val_x,val_y_1hot))

        metric = train.history['val_IoU']             ## I'm using val-metric for sucessive halving evaluation not loss
        return metric[-1]

In [ ]:
from keras.optimizers import Adam
from keras.backend import tensorflow_backend as K

config = hyp.search(define)   ## calling the search function from hyperband


  Current bracket number -  4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="Unet", inputs=Tensor("in..., outputs=Tensor("co...)`



 	 running the config  ['0.005' '0.92' '0.998' 'focal_loss']  for -  1  epochs
Train on 20 samples, validate on 4 samples
Epoch 1/1
20/20 [==============================] - 13s 630ms/step - loss: 13654.8676 - IoU: 0.0946 - val_loss: 11761.2725 - val_IoU: 0.1678

 	 running the config  ['0.005' '0.9' '0.999' 'cross_entropy']  for -  1  epochs
Train on 20 samples, validate on 4 samples
Epoch 1/1
20/20 [==============================] - 2s 107ms/step - loss: 14562.9699 - IoU: 0.0421 - val_loss: 15769.8193 - val_IoU: 0.0019

 	 running the config  ['0.0001' '0.9' '0.997' 'cross_entropy']  for -  1  epochs
Train on 20 samples, validate on 4 samples
Epoch 1/1
20/20 [==============================] - 2s 108ms/step - loss: 13958.5232 - IoU: 0.0497 - val_loss: 12155.7393 - val_IoU: 0.1229

 	 running the config  ['0.008' '0.92' '0.999' 'focal_loss']  for -  1  epochs
Train on 20 samples, validate on 4 samples
Epoch 1/1
20/20 [==============================] - 2s 106ms/step - loss: 12016.6916 -

ResourceExhaustedError: ignored